In [ ]:
import json
import requests
import constants
import time
import random
from tqdm import tqdm

In [ ]:
import sys, os
head = os.path.split(os.path.split(os.getcwd())[0])[0]
sys.path.append(head)

In [ ]:
import crawler.reviews_parser as rp

In [ ]:
#Change this to determine which experiment is loaded
extended = False

#Set data source
if extended:
    source = constants.CRAWL_SOURCE_EXTENDED
else:
    source = constants.CRAWL_SOURCE_CHICAGO
constants.set_crawl_source(source)

In [ ]:
with open(constants.MISSING_MASK_DATA) as fp:
    missing_data = json.load(fp)

In [ ]:
try:
    with open(constants.REPLACEMENT_MASK_DATA,"r") as fp:
        mask_data = json.load(fp)
except:
    mask_data = {}

In [ ]:
for bid, url in tqdm(missing_data.items(), total=len(missing_data)):
    if bid in mask_data:
        if mask_data[bid]['ammenities_need_manual_intervention']:
            print(f"{bid}: Failed last time, trying again")
        else:
            print(f"{bid}: Skipped ({'Fail' if mask_data[bid]['ammenities_need_manual_intervention'] else 'Success'})")
            continue
    response = requests.get(url)
    mask_data[bid] = rp.extract_business_data(response.content.decode())
    print(f"{bid}: {'Fail' if mask_data[bid]['ammenities_need_manual_intervention'] else 'Success'}")
    time.sleep(random.randrange(30,90))

In [ ]:
with open(constants.REPLACEMENT_MASK_DATA,"w+") as fp:
    json.dump(mask_data,fp)

In [ ]:
for bid, data in mask_data.items():
    if data["ammenities_need_manual_intervention"]:
        print(f"{bid}: {missing_data[bid]}")

In [ ]:
missing_data["RsnbR0pXKh27kcckkblJqQ"]